In [45]:
from sklearn.naive_bayes import GaussianNB 
from sklearn import cross_validation
from time import time
import numpy as np
import pandas as pd

In [46]:
train_data = pd.read_csv('train_prep.csv')
test_data = pd.read_csv('test_prep.csv')

In [47]:
param = train_data.ix[:, :-1] # take all rows and all but last column from training data
res = train_data.ix[:, -1] # take all rows and only last column from training data
test_data = test_data.drop('ID', 1) # drop ID column from test data

In [82]:
model = GaussianNB()
cross_validation.cross_val_score(model, param, res, scoring="neg_log_loss")
# cross_validation.cross_val_score(model, param, res, cv=5)

array([-1.13309323, -1.21357119, -1.2124326 ])

In [85]:
# create output file
def create_output_file(data, file_name):
    output = []
    for pred in data:
        oi = [0] * 5
        oi[pred - 1] = 1
        output.append(oi)
    output = pd.DataFrame(output, columns=['Adoption', 'Died', 'Euthanasia', 'Return_to_owner', 'Transfer'])
    output.index.names = ['ID']
    output.index += 1
    output.to_csv(file_name, index_label='ID')

In [86]:
model.fit(param, res) # fit the data
predictions = model.predict(test_data) # make predictions
create_output_file(predictions, 'Gaussian-NB.csv')